In [2]:
!pip3 install flask geopy

'pip3' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import gmaps
import pandas as pd

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [4]:
crime_df = pd.read_csv('Resources/KCPD_Crime_Data.csv')
crime_df.dropna(inplace = True) 
crime_df.head()

,Report_No,Reported_Date,Reported_Time,From_Date,From_Time,To_Date,To_Time,Offense,IBRS,Description,...,Zip Code,Rep_Dist,Area,DVFlag,Involvement,Race,Sex,Age,Firearm Used Flag,Location
384,KC20011765,02/15/2020,2020-02-15T20:03:38.000,02/10/2020,2020-02-10T12:00:00.000,02/15/2020,2020-02-15T15:00:00.000,Stolen Auto,23F,Theft From Motor Vehicle,...,64106.0,PJ0696,CPD,N,VIC,W,M,60.0,False,W 8TH ST and MULBERRY ST\nKANSAS CITY 64106\n(...
385,KC20012815,02/20/2020,2020-02-20T06:55:03.000,02/19/2020,2020-02-19T00:01:00.000,02/20/2020,2020-02-20T06:50:00.000,Stealing from Auto (Theft from Auto),23F,Theft From Motor Vehicle,...,64111.0,PJ3372,CPD,N,VIC,W,F,23.0,False,"4100 OAK ST\nKANSAS CITY 64111\n(39.051476, -..."
398,KC20007266,01/29/2020,2020-01-29T16:25:56.000,01/29/2020,2020-01-29T05:30:00.000,01/29/2020,2020-01-29T16:50:00.000,Burglary (Residential),220,Burglary/Breaking and Entering,...,64129.0,PJ4134,MPD,N,SUS,B,M,22.0,False,7600 SNI-A-BAR RD\nKANSAS CITY 64129\n(39.035...
410,KC20008528,02/03/2020,2020-02-03T12:10:14.000,12/17/2019,2019-12-17T03:52:00.000,02/03/2020,2020-02-03T12:10:00.000,Identity Theft,26F,Identity Theft,...,64151.0,PP0691,NPD,N,VIC,W,M,59.0,False,5700 N HUTSON RD\nKANSAS CITY 64151\n(39.1987...
419,KC20008819,02/04/2020,2020-02-04T11:21:03.000,02/02/2020,2020-02-02T11:34:00.000,02/04/2020,2020-02-04T10:29:00.000,Violation of Ex-Parte Order of Protection,90Z,All Other Offenses,...,64114.0,PJ6193,MPD,Y,VIC,B,F,30.0,False,8700 WORNALL RD\nKANSAS CITY 64114\n(38.97066...


In [5]:
location = crime_df["Location"]= crime_df["Location"].str.split("(", n = 1, expand = True) 
  
location

,0,1
384,W 8TH ST and MULBERRY ST\nKANSAS CITY 64106\n,"39.105094, -94.599678)"
385,4100 OAK ST\nKANSAS CITY 64111\n,"39.051476, -94.581738)"
398,7600 SNI-A-BAR RD\nKANSAS CITY 64129\n,"39.035286, -94.495582)"
410,5700 N HUTSON RD\nKANSAS CITY 64151\n,"39.198787, -94.620837)"
419,8700 WORNALL RD\nKANSAS CITY 64114\n,"38.970664, -94.59528)"
...,...,...
47953,2400 E 11TH ST\nKANSAS CITY 64127\n,"39.100736, -94.553634)"
47978,8500 E 99TH ST\nKANSAS CITY 64134\n,"38.944361, -94.491134)"
47982,12200 N POMONA LN\nKANSAS CITY 64153\n,"39.315115, -94.671276)"
47984,5300 BROOKLYN AVE\nKANSAS CITY 64130\n,"39.030194, -94.559609)"


In [6]:
location = location[1].str.split(",",expand=True)

In [7]:
location[1] = location[1].str.replace(')', '')

In [8]:
renamed= location.rename(columns={0: "Lat", 1:"Lng"})
#renamed.to_csv('Crime_Lat_Lng.csv')
renamed

,Lat,Lng
384,39.105094,-94.599678
385,39.051476,-94.581738
398,39.035286,-94.495582
410,39.198787,-94.620837
419,38.970664,-94.59528
...,...,...
47953,39.100736,-94.553634
47978,38.944361,-94.491134
47982,39.315115,-94.671276
47984,39.030194,-94.559609


In [9]:
coordinates= renamed[["Lat","Lng"]]
coordinates['weight']=1
coordinates

,Lat,Lng,weight
384,39.105094,-94.599678,1
385,39.051476,-94.581738,1
398,39.035286,-94.495582,1
410,39.198787,-94.620837,1
419,38.970664,-94.59528,1
...,...,...,...
47953,39.100736,-94.553634,1
47978,38.944361,-94.491134,1
47982,39.315115,-94.671276,1
47984,39.030194,-94.559609,1


In [10]:
coordinates["Lat"]= pd.to_numeric(coordinates["Lat"])

In [11]:
coordinates["Lng"]= pd.to_numeric(coordinates["Lng"])

In [12]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates[["Lat","Lng"]], weights = coordinates['weight'],
                                 dissipating=False, max_intensity=50,
                                 point_radius=0.01)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))